# Bringing a Gun to a Trainer Fight

## Problem

Uh-oh -- you've been cornered by one of Commander Lambdas elite bunny trainers! Fortunately, you grabbed a beam weapon from an abandoned storeroom while you were running through the station, so you have a chance to fight your way out. But the beam weapon is potentially dangerous to you as well as to the bunny trainers: its beams reflect off walls, meaning you'll have to be very careful where you shoot to avoid bouncing a shot toward yourself!
<br>
Luckily, the beams can only travel a certain maximum distance before becoming too weak to cause damage. You also know that if a beam hits a corner, it will bounce back in exactly the same direction. And of course, if the beam hits either you or the bunny trainer, it will stop immediately (albeit painfully). 
<br>
Write a function solution(dimensions, your_position, trainer_position, distance) that gives an array of 2 integers of the width and height of the room, an array of 2 integers of your x and y coordinates in the room, an array of 2 integers of the trainer's x and y coordinates in the room, and returns an integer of the number of distinct directions that you can fire to hit the elite trainer, given the maximum distance that the beam can travel.
<br>
The room has integer dimensions [1 < x_dim <= 1250, 1 < y_dim <= 1250]. You and the elite trainer are both positioned on the integer lattice at different distinct positions (x, y) inside the room such that [0 < x < x_dim, 0 < y < y_dim]. Finally, the maximum distance that the beam can travel before becoming harmless will be given as an integer 1 < distance <= 10000.
<br>
For example, if you and the elite trainer were positioned in a room with dimensions [3, 2], your_position [1, 1], trainer_position [2, 1], and a maximum shot distance of 4, you could shoot in seven different directions to hit the elite trainer (given as vector bearings from your location): [1, 0], [1, 2], [1, -2], [3, 2], [3, -2], [-3, 2], and [-3, -2]. As specific examples, the shot at bearing [1, 0] is the straight line horizontal shot of distance 1, the shot at bearing [-3, -2] bounces off the left wall and then the bottom wall before hitting the elite trainer with a total shot distance of sqrt(13), and the shot at bearing [1, 2] bounces off just the top wall before hitting the elite trainer with a total shot distance of sqrt(5).

## Solution

In [1]:
from math import atan2

def dist(x1, x2):
    return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2)

def findAngle(base, final):
    angle = atan2(final[1] - base[1], final[0] - base[0])
    return angle

### Find angles, distances and everything for the reflections
def findReflections(x, y, x_size, y_size, number, your_position):
    reflections = []
    for i in range(-number, number + 1, 1):
        reflections_x = []
        for j in range(-number, number + 1, 1):
            trans_x, trans_y = False, False        
            if i % 2 == 0:
                x_final = x
            else:
                x_final = x_size - x
            if j % 2 == 0:
                y_final = y
            else:
                y_final = y_size - y
            fifa_x = x_final + x_size*i
            fifa_y = y_final + y_size*j
            angle = findAngle((your_position[0], your_position[1]), (fifa_x, fifa_y))
            distance = dist((your_position[0], your_position[1]), (fifa_x, fifa_y))
            reflections_x.append((fifa_x, fifa_y, angle, distance))
        reflections.append(reflections_x)
    return reflections

## So, I am proud of this. Many trainers were beaten... And pretty fast
## My logic is to find all the reflections of the rooms around the room in every direction limited by the distance
def solution(dimensions, your_position, trainer_position, distance):
    if your_position == trainer_position:
        return 0
    ## This limits the number of reflections to be found
    number = max(int(distance/(dimensions[0])), int(distance/dimensions[1])) + 1
    
    ## I need reflections of my positions, cos i dont want to be in firing zone
    my_postions = findReflections(your_position[0],
                                    your_position[1],
                                    dimensions[0], 
                                    dimensions[1],
                                    number,
                                    your_position)
    
    ## I need reflections of trainers to attack them
    tr_positions = findReflections(trainer_position[0],
                                    trainer_position[1],
                                    dimensions[0], 
                                    dimensions[1], 
                                    number,
                                    your_position)
    
    final = 0
    midpoint = int(len(tr_positions)/2)
    angles = {}
    angles2 = {}
    count = 0
    x = 1
    dist_sq = distance * distance
    ## Direct shot. Need to check directly.. cos it doesn't work inside :P
    if dist(your_position, trainer_position) <= dist_sq:
        angles[tr_positions[midpoint][midpoint][2]] = tr_positions[midpoint][midpoint][3]
        count += 1
    ## To figure this out, I took a lot of time... Almost got killed by the trainers
    ## So basically I am going round and round and round the reflections and noting down the angles. 
    ## This way, any points with repeat angles will be eliminated
    while x <= number:
        for i in range(-x, x + 1):
            list_pts = [(midpoint + i, midpoint + x),
                        (midpoint + i, midpoint - x),
                        (midpoint + x, midpoint + i),
                        (midpoint - x, midpoint + i)]
            for j in list_pts:
                if my_postions[j[0]][j[1]][2] not in angles2:
                    angles2[my_postions[j[0]][j[1]][2]] = my_postions[j[0]][j[1]][3]
                if tr_positions[j[0]][j[1]][3] <= dist_sq:
                    if (tr_positions[j[0]][j[1]][2] not in angles and 
                        tr_positions[j[0]][j[1]][2] not in angles2):
                        angles[tr_positions[j[0]][j[1]][2]] = tr_positions[j[0]][j[1]][3]
                        count += 1

        x += 1
    return count

## Test Cases

In [2]:
solution([300,275], [150,150], [185,100], 500)

9

In [3]:
solution([3,2], [1,1], [2,1], 4)

7

In [4]:
solution([10,10],[3,3],[4,4],5000) #### It was hard to beat this guy

739323